# Unsupervised Learning Trading Strategy

* Download/Load SP500 stocks prices data.
* Calculate different features and indicators on each stock.
* Aggregate on monthly level and filter top 150 most liquid stocks.
* Calculate Monthly Returns for different time-horizons.
* Download Fama-French Factors and Calculate Rolling Factor Betas.
* For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
* For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization.
* Visualize Portfolio returns and compare to SP500 returns.

* https://github.com/Luchkata/Algorithmic_Trading_Machine_Learning/blob/main/Algorithmic_Trading_Machine_Learning_Quant_Strategies.ipynb

In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

print('df')
print(df)
print('sp500')
print(sp500)
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

df

[***********           23%%                      ]  114 of 503 completed

In [ ]:
# https://help.yahoo.com/kb/SLN28256.html
# Adj closeは該当するすべての株式分割および配当分配を調整した後の終値です。
# データは、証券価格研究センター (CRSP) の標準に準拠し、適切な株式分割および配当乗数を使用して調整されます。

# https://breakingdownfinance.com/finance-topics/risk-management/garman-klass-volatility/
# https://portfolioslab.com/tools/garman-klass
# Garman Klass は、証券の始値、安値、高値、終値を組み込んだボラティリティ推定式です。
# 欠点はあるものの、Garman-Klass 推定値は、時間間隔の開始時と終了時の価格だけでなく、日中の価格の極値も考慮するため、基本的な式よりも効果的です。
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# RSIとは、「Relative Strength Index」の略で、テクニカルチャートのひとつです。日本語に訳すと「相対力指数」になります。買われすぎか、売られすぎかを判断するための指標として利用されています。
# RSIは、過去一定期間の上げ幅（前日比）の合計を、同じ期間の上げ幅の合計と下げ幅の合計を足した数字で割って、100を掛けたものです。
# いくら値上がり、値下がりしたかはRSIでは判断できません。数値は0～100で表され、一般的に70～80％以上で買われすぎ、20～30％以下で売られすぎと判断されます。
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# ボリンジャーバンドとは、移動平均線とその上下2本ずつの標準偏差からなる線の計5本の線で表わされます。英字表記は「Bollinger bands」となります。
# ボリンジャーバンドは統計学を使って作られていて、大まかにいうと、高い確率で＋2σ(標準偏差)と－2σのラインの間で価格は動くだろうという予測をもとに将来の価格の動きを予測するために使います。
# なお、統計学上、＋2σと－2σの間に収まる確率は95.45%とされています。
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

# 「相場の変動率」を解析する指標（計算式）が「ATR（アベレージトゥルーレンジ)」です。 相場の変動が大きい傾向なのか小さい傾向なのかを分析する場合に有効です。 
# 「当日高値-当日安値」「当日高値-前日終値」「当日安値-前日終値」の3つのうち最大の値幅(マド明けを含む最大値幅の計測)を当日の「真の値幅(トゥルーレンジ)」と呼び、
# この「真の値幅」の移動平均線がATR(アベレージトゥルーレンジ)です。
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
             low=stock_data['low'],
             close=stock_data['close'],
             length=14)
    return atr.sub(atr.mean()).div(atr.std())
    
# 今回と関係ないけど日本語でATR出したりと似たようなことしている記事あった　　https://qiita.com/__x__/items/ed91e995aec21ac89c8b
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# MACD（通称マックディー）は、移動平均の発展版で、更に売買シグナルにおいて精度を高くした、トレンド分析の中でも人気のある指標の一つです。
# 「移動平均収束拡散」又は「移動平均収束乖離」などとも呼ばれています。トレンド形成時に威力を発揮するため、逆にボックス相場に弱いのが特徴です。
# 主にMACDとMACDシグナルという2本のラインの交差を売買のタイミングとして用いられることが多く、初心者の方でもシグナルの発見が容易です。
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# １００万株取引されることがわかっているので1e6で割る(?)
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

# Aggregate to monthly level and filter top 150 most liquid stocks for each month.

トレーニング時間を短縮し、機能や戦略を実験するために、毎日の営業データを月末の頻度に変換します。
To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [ ]:
# ここから列の並び順が動画と違う？書いてる内容は同じなはずだが・・・
# doller volumeは各銘柄の月全体の平均調整後価格を出したいらしい

# 別にuniqueにしなくても良さそうだけどしてるみたい。
# 全データが日時なため、日時→月次にしたいらしい。
# OHLCとは（Open/High/Low/Close）の省略表記で、ローソク足の価格データセット（始値・高値・安値・終値）である。ここに出来高（Volume）を加え、OHLCVで提供されることも多い。
# OHLCは要らないらしくテクニカル指標を指定したいらしい。インジケーターは列を指定する必要があるのでインジケータ
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

# 時系列データを元データより高い頻度または低い頻度で再度サンプリングすることをリサンプリングと呼ぶ。MはMonthly
# 今は日時がスタックしているdfをticker単位でunstackして、dollar_volumeとテクニカル指標をそれぞれ月次にresampleして、dollar_volumeは平均を出しつつ、
# テクニカル指標はlastで月末の値をシンプルに指定しつつstackし直す
# dropnaは欠損値を落とす
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

In [ ]:
# Calculate 5-year rolling average of dollar volume for each stocks before filtering.
# それを下に各月のもっとも流動性の高い１５０種類をピックアップしたいらしい

# ローリング平均では、データ・セットの平均が継続的に更新され、その時点までのデータすべてが算入されます。
# 例えば、2012 年 3 月の返品数量のローリング平均は、1 月、2 月、3 月の返品数量を加算し、その合計を 3 で除算して計算されます。
# locは行名もしくは列名を指定することで特定の値を抽出できます。（行名や列名をラベルと置き換えて頂いても問題ありません。）
# ilocはindexを指定することで特定の値を抽出できます。つまり、行、列を番号（数字が０のインデックス）で指定します。

data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

# 4.calculate monthly returns for different time horizons as features
To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.
例えば、モメンタムパターンを反映する時系列ダイナミクスを捉えるために、ラグで特定される様々な月次期間のリターンを計算する、 
.pct_change(lag)という方法で過去のリターンを計算する、すなわち、様々な月次期間のリターンをラグで識別する。
↑訳してもわからん

時間軸で分けたいろいろなデータを取ることで該当銘柄のモメンタムを見れるらしい

In [ ]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        # diff()がA - Bで差分を算出するのに対し、pct_change()は(A - B) / Bで変化率を算出する。
        # 3^4 = 3 ** 4 = pow(3,4) その結果は冪 (べき、英: power) と呼ばれる。表現の揺れにより同じ概念は日本語で「累乗」とも表現されている
        # 四分位数とは、データを大きさの順に並べて、個数を４等分できる値のことです。森からうさぎを40羽連れてきて、体の大きさ順に並べます。
        # うさぎをピッタリ４等分できる線は、10と11番目の間、20と21番目の間、30と31番目の間の線になります。これが四分位数です。
        # pipe.clip 外れ値を四分位範囲を用いてクリップする https://ensekitt.hatenablog.com/entry/2018/02/13/200000
        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

# 5. Download Fama-French Factors and Calculate Rolling Factor Betas
Fama-French 5ファクターモデルとは、Fama-French 3ファクターモデルに、収益性（Profitability）と投資（Investment）のファクターを追加したモデルであり、下記式であらわされるモデルである。
We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.
本論文では、線形回帰を用いて一般的なリスク要因に対する資産のエクスポージャーを推定するためのFama-Frenchデータを紹介する。
市場リスク、規模、バリュー、営業収益性、投資の5つのFama-Frenchファクターは、経験的に資産リターンを説明することが示されており、ポートフォリオのリスク／リターン・プロファイルを評価するために一般的に使用されている。したがって、過去のファクター・エクスポージャーを財務的特徴としてモデルに含めることは自然なことです。
pandas-datareader を使って過去のファクターリターンにアクセスし、RollingOLS のローリング線形回帰を使って過去のエクスポージャを推定することができます。

SMB (Small Minus Big) is the average return on the nine small stock portfolios minus the average return on the nine big stock portfolios,

HML (High Minus Low) is the average return on the two value portfolios minus the average return on the two growth portfolios,

RMW (Robust Minus Weak) is the average return on the two robust operating profitability portfolios minus the average return on the two weak operating profitability portfolios,

CMA (Conservative Minus Aggressive) is the average return on the two conservative investment portfolios minus the average return on the two aggressive investment portfolios,

Rm-Rf, the excess return on the market, value-weight return of all CRSP firms incorporated in the US and listed on the NYSE, AMEX, or NASDAQ that have a CRSP share code of 10 or 11 at the beginning of month t, good shares and price data at the beginning of t, and good return data for t minus the one-month Treasury bill rate (from Ibbotson Associates).

SMB（Small Minus Big）は、9つの小型株ポートフォリオの平均リターンから 9 つの大型株ポートフォリオの平均リターンを差し引いたものです、
HML（High Minus Low）は、2つのバリュー・ポートフォリオの平均リターンから2つのグロース・ポートフォリオの平均リターンを差し引いたものです、
RMW（Robust Minus Weak）は、2つの強固な営業収益性ポートフォリオの平均リターンから2つの脆弱な営業収益性ポートフォリオの平均リターンを差し引いたものです、
CMA（Conservative Minus Aggressive）は、2つの保守的投資ポートフォリオの平均リターンから2つの積極的投資ポートフォリオの平均リターンを引いたものである、
Rm-Rfは、米国で設立され、NYSE、AMEX、またはNASDAQに上場しているすべてのCRSP企業のうち、t月初にCRSPの株式コードが10または11であり、t月初に良好な株式と価格のデータ、およびtの良好なリターンのデータから1ヶ月物国庫短期証券レートを差し引いた市場、バリューウェイトリターンの超過リターン（Ibbotson Associatesより）。

In [ ]:
# Kenneth French's data library：ケネス・R・フレンチのHPがpandas_datareaderに対応しているっぽい
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
               'famafrench',
               start='2010'
              )[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'date'
factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data


In [ ]:
# 属性sizeで全要素数、属性shapeで形状（行数、列数）が取得できる。要素数が少ない銘柄を除外しているっぽい
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]
# ここで「key1 が A の行だけフィルタリングしたい」という場合、どうすればいいか。→pandas.MultiIndex.get_level_values というメソッドを使うと、Multi-Index の値にアクセスできる。
# isin()メソッドは、リスト内の値が別のリストに含まれているかどうかをチェックするために利用します。リストの値をカンマで区切って指定し、リストの値が含まれている場合はTrue、含まれていない場合はFalseを返します。
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]
factor_data


In [ ]:
# 最小二乗法 (OLS) は、最もよく知られている回帰分析手法です。これは、すべての空間回帰分析の開始点でもあります。
# 理解または予測しようとしている変数またはプロセスのグローバル モデルを作成し、そのプロセスを表す単一の回帰方程式を作成します。
# Rolling OLS applies OLS across a fixed windows of observations and then rolls (moves or slides) the window across the data set. 
# They key parameter is window which determines the number of observations used in each OLS regression. 
# By default, RollingOLS drops missing values in the window and so will estimate the model using the available data points.
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas 

In [ ]:
# 計算したbetasをもともとのdataに混ぜたいが、rolling factorは次の月末の結果を計算しておりそのままだと１ヶ月のズレが生じてしまうため、betasを一ヶ月早めてから(shift())混ぜる
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
# shift()デフォルトでは下方向に1行ずれる。行数はそのままなので、最後の行のデータは削除される。
data = (data.join(betas.groupby('ticker').shift()))

# 欠損値は平均値で埋める
data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()

# 6. For each month fit a K-Means Clustering Algorithm to group similar assets based on their features

In [ ]:
# hypothesis: rsi 70 stock has stock momentum
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html 
# If an array is passed, it should be of shape (n_clusters, n_features) and gives the initial centers.
# centroid 【名】 《物理》質量中心
target_rsi_values = [30, 45, 55, 70]

initial_centeroids = np.zeros((len(target_rsi_values), 18))

# 6=rsi
initial_centeroids[:,6] = target_rsi_values

initial_centeroids

In [ ]:
from sklearn.cluster import KMeans

def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=4,
                           random_state=0,
                           init=initial_centeroids).fit(df).labels_
    return df

data = data.dropna().groupby('date', group_keys=False).apply(get_clusters)

data

In [ ]:
def plot_clusters(data):

    cluster_0 = data[data['cluster']==0]
    cluster_1 = data[data['cluster']==1]
    cluster_2 = data[data['cluster']==2]
    cluster_3 = data[data['cluster']==3]

    # scatter first column=atr, second column=rsi
    # 動画と自分の表の列の並び順が違うので書き換えちゃう
    plt.scatter(cluster_0.iloc[:,0] , cluster_0.iloc[:,6] , color = 'red', label='cluster 0')
    plt.scatter(cluster_1.iloc[:,0] , cluster_1.iloc[:,6] , color = 'green', label='cluster 1')
    plt.scatter(cluster_2.iloc[:,0] , cluster_2.iloc[:,6] , color = 'blue', label='cluster 2')
    plt.scatter(cluster_3.iloc[:,0] , cluster_3.iloc[:,6] , color = 'black', label='cluster 3')
    # plt.scatter(cluster_0.iloc[:,5] , cluster_0.iloc[:,1] , color = 'red', label='cluster 0')
    # plt.scatter(cluster_1.iloc[:,5] , cluster_1.iloc[:,1] , color = 'green', label='cluster 1')
    # plt.scatter(cluster_2.iloc[:,5] , cluster_2.iloc[:,1] , color = 'blue', label='cluster 2')
    # plt.scatter(cluster_3.iloc[:,5] , cluster_3.iloc[:,1] , color = 'black', label='cluster 3')
    
    plt.legend()
    plt.show()
    return

In [ ]:
# 動画と指標の数値がぜんぜん違うので、多分カラム順番が間違っている
plt.style.use('ggplot')

for i in data.index.get_level_values('date').unique().tolist():

    g = data.xs(i,level=0)
    plt.title(f'Date {i}')
    plot_clusters(g)
    

# 7. For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization
First we will filter only stocks corresponding to the cluster we choose based on our hypothesis.

Momentum is persistent and my idea would be that stocks clustered around RSI 70 centroid should continue to outperform in the following month - thus I would select stocks corresponding to cluster 3.

In [ ]:
filtered_df = data[data['cluster']==3].copy()

filtered_df = filtered_df.reset_index(level=1)

# 日付が月の最終日になっているので+1日して月初に変えたい
filtered_df.index = filtered_df.index+pd.DateOffset(1)

# 日付と重みをもたせる
filtered_df = filtered_df.reset_index().set_index(['date', 'ticker'])

dates = filtered_df.index.get_level_values('date').unique().tolist()

fixed_dates = {}

for d in dates:
    
    fixed_dates[d.strftime('%Y-%m-%d')] = filtered_df.xs(d, level=0).index.tolist()

# 月ごとに投資するべき銘柄のリストを出力できる
fixed_dates

## Define portfolio optimization function
We will define a function which optimizes portfolio weights using PyPortfolioOpt package and EfficientFrontier optimizer to maximize the sharpe ratio.

To optimize the weights of a given portfolio we would need to supply last 1 year prices to the function.

Apply signle stock weight bounds constraint for diversification (minimum half of equaly weight and maximum 10% of portfolio).

In [ ]:
# それぞれの銘柄にどれくらいの割合で投資すべきか重み付けをしたい
# 効率的フロンティア　（Efficient Frontier）とは、分散投資を実施したときに実現するポートフォリオの中で、あるリスクの水準で最大のリターンを獲得できるポートフォリオの集合のことを指す。

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

def optimize_weights(prices, lower_bound=0):
    
    returns = expected_returns.mean_historical_return(prices=prices,
                                                      frequency=252)

    # sample_covは標本分散共分散行列です。
    # 分散共分散行列とは，分散（散らばり具合を表す指標）の概念を多次元確率変数に拡張して行列としたもの。データの散らばり具合や相関という情報を集約したものです！
    cov = risk_models.sample_cov(prices=prices,
                                 frequency=252)

    # solver (string) – which SCIPY solver to use, e.g “SLSQP”, “COBYLA”, “BFGS”. User beware: different optimizers require different inputs.
    # シンプレックス法による制約ソルバー（本章では， SCS(simplex constraint solver) と呼ぶ）.
    # ソルバーについて(エクセルの記事だけど・・・)
    # エクセルのソルバー機能を使用して線形計画問題(Linear Problem)を解く方法について説明します。
    # 線形計画問題は、限られたリソースを最適に配分し、行動を最適化するための強力なツールです。そして、エクセルのソルバー機能を使えば、これらの問題を簡単に解くことができます。
    ef = EfficientFrontier(expected_returns=returns,
                           cov_matrix=cov,
                           weight_bounds=(lower_bound, .1),
                           solver='SCS')
    
    weights = ef.max_sharpe()
    
    return ef.clean_weights()

In [ ]:
stocks = data.index.get_level_values('ticker').unique().tolist()

new_df = yf.download(tickers=stocks,
                     start=data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                     end=data.index.get_level_values('date').unique()[-1])

new_df

In [ ]:
returns_dataframe = np.log(new_df['Adj Close']).diff()
portfolio_df = pd.DataFrame()

for start_date in fixed_dates.keys():

    try:
    
        end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')
        
        cols = fixed_dates[start_date]
    
        optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')
        optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d') 

        optimization_df = new_df[optimization_start_date:optimization_end_date]['Adj Close'][cols]

        success = False

        try:
    
            weights = optimize_weights(prices=optimization_df,
                                   lower_bound=round(1/(len(optimization_df.columns)*2),3))
        
            weights = pd.DataFrame(weights, index=pd.Series(0))

            success = True
        except:
            print('Max Sharpe Optimization failed for {start_date}, Continuing with Equal-Weights')
                   
        if success==False:
            weights = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                     index=optimization_df.columns.tolist(),
                                     columns=pd.Series(0)).T
        
        temp_df = returns_dataframe[start_date:end_date]

        # [Date, Index]だとエラーになるので、[Date, Ticker]にした
        temp_df = temp_df.stack().to_frame('return').reset_index(level=0)\
                   .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
                          left_index=True,
                          right_index=True)\
                   .reset_index().set_index(['Date', 'Ticker']).unstack().stack()
        
        temp_df.index.names = ['date', 'ticker']

        temp_df['weighted_return'] = temp_df['return']*temp_df['weight']

        temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')

        portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)
    
    except Exception as e:
        print(e)

portfolio_df = portfolio_df.drop_duplicates()

portfolio_df


In [ ]:
# # len(optimization_df.columns)が動画では１０になるのに自分でやると５３になるのはなぜ
# weights = optimize_weights(prices=optimization_df,
#                            lower_bound=round(1/(len(optimization_df.columns)*2),3))
# weights = pd.DataFrame(weights, index=pd.Series(0))

# エラーになる
# temp_df = returns_dataframe['2017-11-01':'2017-11-30']

# temp_df.stack().to_frame('return').reset_index(level=0)\
#     .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
#           left_index=True,
#           right_index=True)\
#     .reset_index().set_index(['Date', 'index']).unstack().stack()

# 8. Visualize Portfolio returns and compare to SP500 returns.

In [ ]:
spy = yf.download(tickers='SPY',
                 start='2015-01-01',
                 end=dt.date.today())
spy_ret = np.log(spy[['Adj Close']]).diff().dropna().rename({'Adj Close':'SPY BuyHold'}, axis=1)

portfolio_df = portfolio_df.merge(spy_ret,
                                 left_index=True,
                                 right_index=True)

portfolio_df

In [ ]:
# 負けとるやん
import matplotlib.ticker as mtick

plt.style.use('ggplot')

portfolio_cumulative_return = np.exp(np.log1p(portfolio_df).cumsum())-1

portfolio_cumulative_return[:'2023-09-29'].plot(figsize=(16,6))

plt.title('Unsupervised Learning Trading Strategy Returns Over Time')

plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1))

plt.ylabel('Return')

plt.show()